In [ ]:
import os
import numpy as np
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D
from keras.regularizers import l2
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras.layers import Input, GlobalAveragePooling2D
from keras import models
from keras.models import Model
from tensorflow_model_optimization.quantization.keras import vitis_quantize

shape_x = 64
shape_y = 64
# X_train=np.random.randint(0,255,size=[60,32,32,1])
# Y_train=np.random.randint(1,11,size=[60,])
# X_test=np.random.randint(0,255,size=[60,32,32,1])
# Y_test=np.random.randint(1,11,size=[60,])
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')
# X_train /= 255.0
# X_test /= 255.0
input_x = 64
input_y = 64
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print('Training data: {}, {}'.format(x_train.shape, y_train.shape))
print('Test data: {}, {}'.format(x_test.shape, y_test.shape))
x_train = x_train[0:60]
y_train = y_train[0:60]
x_test = x_test[0:10]
y_test = y_test[0:10]

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)
print(y_train.shape)
x_train = tf.image.resize(x_train, [64,64])
x_test = tf.image.resize(x_test, [64,64])
print(x_train.shape)
input_img = Input(shape=(shape_x, shape_y, 1))

### 1st layer
layer_1 = Conv2D(10, (1,1), padding='same', activation='relu')(input_img)
layer_1 = Conv2D(10, (3,3), padding='same', activation='relu')(layer_1)

layer_2 = Conv2D(10, (1,1), padding='same', activation='relu')(input_img)
layer_2 = Conv2D(10, (5,5), padding='same', activation='relu')(layer_2)

layer_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
layer_3 = Conv2D(10, (1,1), padding='same', activation='relu')(layer_3)

mid_1 = tf.keras.layers.concatenate([layer_1, layer_2, layer_3], axis = 3)
flat_1 = Flatten()(mid_1)

dense_1 = Dense(1200, activation='relu')(flat_1)
dense_2 = Dense(400, activation='relu')(dense_1)
dense_3 = Dense(150, activation='relu')(dense_2)
output = Dense(10, activation='softmax')(dense_3)
model = Model(input_img, output)
model.summary()
model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy", 
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=1)
model.get_weights()[0].dtype
quantizer = vitis_quantize.VitisQuantizer(model)
quantized_model = quantizer.quantize_model(calib_dataset = x_test[1:8], weight_bit=8, activation_bit=8)
quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
quantized_model.save('tf2_mnist_classifier_quantized.h5')
!vai_c_tensorflow2 \
        --model ./tf2_mnist_classifier_quantized.h5 \
        --arch ./arch.json \
        --output_dir ./xmodel_inception \
        --net_name deploy
os.rename("./xmodel_inception/deploy.xmodel","./xmodel_inception/mnist_classifier_vgg16_{input_x}_{input_y}.xmodel".format(input_x=input_x,input_y=input_y))

Training data: (60000, 28, 28), (60000,)
Test data: (10000, 28, 28), (10000,)
(60,)
(60, 64, 64, 1)
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_30 (Conv2D)              (None, 64, 64, 10)   20          input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 64, 64, 10)   20          input_7[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 64, 64, 1)    0           input_7[0][0]             